In [1]:
!pip install google-api-python-client
!pip install google-cloud-vision
!pip install fsspec
!pip install gcsfs
import csv
import datetime
import io
import json
import pprint

from google.api_core import retry
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient import discovery
from googleapiclient import http
from pandas.io.json import json_normalize
from google.colab import auth
from google.cloud import bigquery
from google.cloud import storage
auth.authenticate_user()
import pandas as pd
import requests


print('Successfully imported Python libraries!')

In [ ]:
import time

In [ ]:
API_SCOPES = ['https://www.googleapis.com/auth/doubleclickbidmanager',
            'https://www.googleapis.com/auth/display-video',
            'https://www.googleapis.com/auth/devstorage.read_only',
            'https://www.googleapis.com/auth/cloud-vision']
            

In [2]:
flow = InstalledAppFlow.from_client_config(df, API_SCOPES)
credentials = flow.run_console()
print('Success!')

# Build DBM Read API service object
dbm_service = discovery.build(
    'doubleclickbidmanager', 'v1.1', credentials=credentials)
print('DBM API service object created')

# Build Google Cloud Storage Read API service object
gcs_service = discovery.build('storage', 'v1', credentials=credentials)
print('GCS service object created')

# Create Display Video API service object
display_video_service = discovery.build(
    'displayvideo', 'v1', credentials=credentials)

In [ ]:
PARTNER_ID = ''  #@param {type:"string"}
ADVERTISER_ID = ''  #@param {type:"string"}
CAMPAIGN_ID = ''  #@param {type:"string"}

# For use with legacy DBM API
SDF_VERSION = '5.3'  #@param {type:"string"}

# For use with DV360 API
SDF_VERSION_DV360 = 'SDF_VERSION_5_3'  #@param {type:"string"}

print('DV360 settings saved!')

DV360 settings saved!


In [3]:
request = display_video_service.advertisers().lineItems().list(
    advertiserId=ADVERTISER_ID,
    filter='entityStatus="ENTITY_STATUS_ACTIVE"',
    pageSize=1
    )

response = request.execute()

# Check if response is empty.
if not response:
  print('Advertiser has no active Line Items')
else:
  pprint.pprint(response['lineItems'])

In [ ]:
def get_active_lineitems(ADVERTISER_ID, CAMPAIGN_ID):
  """Returns list of Lineitems with active status."""
  list_lineitems = display_video_service.advertisers().lineItems().list(
      advertiserId=ADVERTISER_ID,
      filter=f'entityStatus="ENTITY_STATUS_ACTIVE" AND campaignId="{CAMPAIGN_ID}"',
      fields='lineItems(lineItemId,displayName)'  # Return only two fields
  ).execute()

  active_lineitems = [li['lineItemId'] for li in list_lineitems['lineItems']]
  return active_lineitems

In [ ]:
def create_insertion_order(parent_campaign_id, new_io_name):
  """Creates a new DV360 insertion order object."""

  # Define our new Insertion Order boilerplate
  new_insertion_order = {
      'campaignId': parent_campaign_id,
      'displayName': new_io_name,  # Define naming convention 
      'entityStatus': 'ENTITY_STATUS_DRAFT',
      'pacing': {
          'pacingPeriod': 'PACING_PERIOD_DAILY',
          'pacingType': 'PACING_TYPE_EVEN',
          'dailyMaxMicros': '1000000'  # Equiv to $1 or local currency
      },
      'frequencyCap': {
          'unlimited': False,
          'timeUnit': 'TIME_UNIT_MONTHS',
          'timeUnitCount': 1,
          'maxImpressions': 5
      },
      'performanceGoal': {
          'performanceGoalType': 'PERFORMANCE_GOAL_TYPE_CPC',
          'performanceGoalAmountMicros': '1000000',  # $1 CPM/CPC target
      },
      'bidStrategy': {
          'fixedBid': {
              'bidAmountMicros': '0'
          },
      },
      'budget': {
          'automationType':
              'INSERTION_ORDER_AUTOMATION_TYPE_NONE',
          'budgetUnit':
              'BUDGET_UNIT_CURRENCY',
          'budgetSegments': [{
              'budgetAmountMicros':
                  '30000000',  # Equiv to $30 or local currency
              'description': 'My first segment',
              'dateRange': {
                  'startDate': {
                      'year': year,
                      'month': month,
                      'day': day
                  },
                  'endDate': {
                      'year': year_plus30,
                      'month': month_plus30,
                      'day': day_plus30
                  }
              }
          }]
      }
  }

  # API create() request to generate new Insertion Order
  newinsertionorder_request = display_video_service.advertisers(
  ).insertionOrders().create(
      advertiserId=ADVERTISER_ID, body=new_insertion_order).execute()

  # Define patch to activate new Insertion Order afer creation
  patch = {
      'entityStatus': 'ENTITY_STATUS_ACTIVE',
  }

  # API patch() request
  display_video_service.advertisers().insertionOrders().patch(
      advertiserId=ADVERTISER_ID,
      insertionOrderId=newinsertionorder_request['insertionOrderId'],
      updateMask='entityStatus',
      body=patch).execute()

  print(newinsertionorder_request)
  return newinsertionorder_request



In [6]:
get_active_lineitems(ADVERTISER_ID,CAMPAIGN_ID)

In [ ]:
def pause_campaign(campaign_id):
  """Updates DV360 campaign object status to paused."""

  patch = {'entityStatus': 'ENTITY_STATUS_PAUSED'}

  if campaign_id in protected_campaigns:
    print(f'Campaign ID {campaign_id} not paused (protected campaign)')
  else:
    display_video_service.advertisers().campaigns().patch(
        advertiserId=ADVERTISER_ID,
        campaignId=campaign_id,
        updateMask='entityStatus',
        body=patch).execute()
    print(f'Campaign ID {campaign_id} successfully paused')

In [ ]:
bqclient = bigquery.Client(project=)
query_string = """
Pull Traffic Data from BigQuery
"""
job = bqclient.query(query_string)

df1 = job.result().to_dataframe()
df1

,day,city,names,lowimpact_traffic,minor_traffic,moderate_traffic,serious_traffic
0,2021-08-17,Auckland,['Northern Motorway'],None,3.0,NaN,None
1,2021-08-17,Auckland,['Northwestern Motorway'],None,1.0,NaN,None
2,2021-08-17,Christchurch,['Christchurch Southern Motorway'],None,1.0,1.0,None
3,2021-08-17,Auckland,['Southern Motorway'],None,NaN,2.0,None


In [7]:
def runBQ(self):
  bqclient = bigquery.Client(project=)
  query_string = """
    Pull Traffic Data from BigQuery
  """
  job = bqclient.query(query_string)

  df1 = job.result().to_dataframe()
  df1.loc[df1['names'] == "['Northwestern Motorway']", 'lineItemId'] = ""
  df1.loc[df1['names'] == "['Northern Motorway']", 'lineItemId'] = ""
  df1.loc[df1['names'] == "['Southwestern Motorway']", 'lineItemId'] = ""
  df1.loc[df1['names'] == "['Southern Motorway']", 'lineItemId'] = ""
  df1.loc[df1['names'] == "['Christchurch Southern Motorway']", 'lineItemId'] = ""
  df1 = df1.fillna(value=0)
  df1 = df1.astype({'lowimpact_traffic':'int64','minor_traffic':'int64','moderate_traffic':'int64','serious_traffic':'int64'})
  return df1

In [8]:
df1 = runBQ(self=True)
df1

In [9]:
refresh_line_items(2100)

In [ ]:
def runLineItems(dataframe):
  def activate_lineitem(lineitem_id):
      """Updates DV360 lineitem object status to active."""

      patch = {'entityStatus': 'ENTITY_STATUS_ACTIVE'}

      display_video_service.advertisers().lineItems().patch(
      lineItemId=lineitem_id,
      advertiserId=ADVERTISER_ID,
      updateMask='entityStatus',
      body=patch).execute()
      return print(f'Lineitem ID {lineitem_id} is active')

  def pause_lineitem(lineitem_id):
      """Updates DV360 lineitem object status to active."""

      patch = {'entityStatus': 'ENTITY_STATUS_PAUSED'}

      display_video_service.advertisers().lineItems().patch(
      lineItemId=lineitem_id,
      advertiserId=ADVERTISER_ID,
      updateMask='entityStatus',
      body=patch).execute()
      return print(f'Lineitem ID {lineitem_id} is paused')

  for x in dataframe.itertuples():
    if (x[6] != 0) or (x[7] != 0):
      activate_lineitem(x[8])
    else:
      pause_lineitem(x[8])
  return 'Line Items Updated'

In [ ]:
def refresh_line_items(interval):
    while True:
      df1 = runBQ(self=True)
      runLineItems(df1)
      time.sleep(interval)

In [10]:
df1